# **Inter-annotator agreement**
## **Inleiding**
Bij wetenschappelijk onderzoek komt er vaak geannoteerde data kijken. Zeker in het domein van de computationele taalkunde zijn annotaties volledig ingeburgerd. Om bijvoorbeeld aan de computer te leren hoe hij spam kan onderscheiden van normale e-mails hebben we eerst een dataset nodig waarin een hele reeks e-mails gelabeld zijn als "spam" of "normaal". Op basis van deze dataset kan dan een *machine learning*-algoritme getraind worden dat voor nieuwe e-mails (die dus niet in de dataset voorkomen) voorspelt of de mail spam dan wel normaal is. Een belangrijk aspect bij het bouwen van geannoteerde datasets is nagaan hoe goed de annotatoren onderling overeenkomen. Formules die deze mate van overeenkomst in een numerieke waarde omzetten worden ***inter-annotator agreement metrics*** (afgekort als IAA) of ***inter-rater reliability metrics*** (afgekort als IRR) genoemd. Deze notebook laat je toe om de IAA/IRR te berekenen op je eigen geannoteerde dataset (om praktische redenen zullen we vanaf hier enkel nog de term "IAA" vermelden).

**TIP**: Uitgebreide informatie rond IAA kan je vinden in de paper "[Computing Inter-Rater Reliability for Observational Data: An Overview and Tutorial](https://pmc.ncbi.nlm.nih.gov/articles/PMC3402032/)" van Kevin A. Hallgren.

## **Keuze van *metric***
Een eerste belangrijke beslissing bij het berekenen van de IAA is de keuze van IAA-*metric*. Deze notebook biedt drie verschillende formules aan, die in principe (bijna) alle soorten geannoteerde data kunnen dekken: Cohen's kappa (κ), Fleiss' κ, en Krippendorff's alfa (α). Voor de keuze van IAA-*metric* volgt de notebook het volgende keuzeschema:

![](https://drive.google.com/uc?export=view&id=1rb9dWrJRVg3AInyEe9khUXsskLGK0XTK)

*(Bron: [GitHub-repository](https://github.com/Hai-Lina/Before-model-training-Crafting-high-quality-text-annotations-for-machine-learning/tree/main) van Alina Herderich)*


Wijzig in onderstaande codecel de waarden van *False* naar *True* op basis van de inhoud van je geannoteerde dataset. Klik daarna op de "play"-knop links in de cel. De beschrijvingen voor elke variabele (die overeenkomen met de *nodes* in het diagram) staan hieronder beschreven:


*   `multilabel`
    * Mochten de annotatoren slechts één label/score toekennen aan elk te annoteren item? → *False*
    * Mochten de annotatoren verschillende labels/scores toekennen aan elk te annotatoren item? → *True*
*   `ontbrekende_annotaties`
    * Hebben alle annotatoren aan elk van de items een label/score gegeven? → *False*
    * Bevat de dataset items waaraan niet elk van de annotatoren een label/score heeft gegeven? → *True*
*   `categorisch`
    * Zit er een volgorde in de labels/scores waartussen de annotatoren konden kiezen? → *False* (bv. Likertschaal en leeftijd)
    * Zijn de labels/scores waartussen de annotatoren konden kiezen categorisch van aard? → *True* (bv. "ja" versus "nee" en gender)
*   `twee_annotatoren`
    * Zijn er drie of meer personen die de dataset hebben geannoteerd? → *False*
    * Zijn er exact twee personen die de dataset hebben geannoteerd? → *True*

Als je voor `multilabel` *True* aanduidt, wijzig dan ook de waarde voor de volgende *node*:


*   `per_label`
    * Wil je de IAA berekenen voor de volledige taak als geheel? → *False*
    * Wil je de IAA berekenen voor elk van de labels/scores in de dataset afzonderlijk? → *True*

Als je voor `categorisch` *False* aanduidt, geef dan ook aan welk soort ordinale labels/scores je dataset bevat in de volgende bijkomende variabele:

*   `soort_ordinaal`
    * Kunnen de labels/scores gerangschikt worden, evenredig verdeeld worden, en hebben ze een absoluut nulpunt? → "ratio" (bv. leeftijd en temperatuur in Kelvin)
    * Kunnen de labels/scores gerangschikt en evenredig verdeeld worden? → "interval" (bv. scores op IQ-test en temperatuur in Celsius)
    * Kunnen de labels/scores gerangschikt worden? → "ordinal" (bv. Likertschaal en ERK-schaal voor taalbeheersingniveaus)
    * (Zijn de labels/scores categorisch van aard, dan moet `soort_ordinaal` de waarde *None* krijgen.)

In [ ]:
# WIJZIG DE WAARDES VAN ONDERSTAANDE VARIABELEN OP BASIS VAN DE INHOUD VAN JE GEANNOTEERDE DATASET

# vier hoofdvariabelen
multilabel = False
ontbrekende_annotaties = False
categorisch = False
twee_annotatoren = False

# subvariabele voor `multilabel`
per_label = False

# subvariabele voor `categorisch`
soort_ordinaal = None

**OPMERKING**: Als je waarden in een codecel wijzigt, moet je opnieuw op de "play"-knop van alle cellen klikken om er zeker van te zijn dat deze wijzingen meegenomen worden in de volledige notebook.

Op basis van de waardes die je aan de variabelen in de codecel hierboven hebt toegekend, passen we nu het keuzeschema toe. Klik op de "play"-knop van de cel hieronder. Als resultaat zal je onderaan te zien krijgen welke IAA-*metric(s)* van toepassing zijn op jouw data.

In [ ]:
# keuzediagram toepassen
if ontbrekende_annotaties:
  l_metrics = ["krippendorffs_alfa"]

elif not categorisch:
  l_metrics = ["krippendorffs_alfa"]

elif multilabel and not per_label:
  l_metrics = ["krippendorffs_alfa"]

elif multilabel and per_label:

  if twee_annotatoren:
    l_metrics = ["cohens_kappa"]
  else:
    l_metrics = ["fleiss_kappa", "krippendorffs_alfa"]

elif categorisch:

  if twee_annotatoren:
    l_metrics = ["cohens_kappa"]
  else:
    l_metrics = ["fleiss_kappa", "krippendorffs_alfa"]

else:
  raise Exception("Ongeldige combinatie van waardes voor de variabelen.")

# uiteindelijke metric(s) printen op het scherm
print(f"Op basis van de waardes die je hebt toegekend aan de variabelen is het mogelijk om volgende IAA-metric(s) te berekenen op je dataset: {l_metrics}.")

## **Inlezen van dataset**
Nu we onze IAA-*metric(s)* gekozen hebben, kunnen we overgaan naar de volgende stap: het inlezen van onze dataset. Hiervoor dien je de geannoteerde data volgens het formaat geïllustreerd in Tabel 1 in een xlsx-bestand (bv. in Microsoft Excel) op te slaan en te uploaden naar dezelfde map als deze notebook. Om je xlsx-bestand op te laden klik je eerst op het onderste icoontje (dat van een map) in de linkertoolbar en vervolgens op het linkericoontje (dat van een bestand met daarin een pijl) in de toolbar die bovenaan verschijnt. Daarna kan je via de bestandsverkenner van je apparaat de dataset uploaden naar de tijdelijke sessie-opslag van deze notebook.

|ID_item |ANNOTATOR_1|ANNOTATOR_2|[...]|ANNOTATOR_*n*|
|:------:|:---------:|:---------:|:---:|:-----------:|
|item_1  |label/score|label/score|[...]|label/score  |
|item_2  |label/score|label/score|[...]|label/score  |
|[...]   |[...]      |[...]      |[...]|[...]        |
|item_*n*|label/score|label/score|[...]|label/score  |

*Tabel 1: Illustratie formaat dataset.*

De ID's van de items vervang je door je eigen ID's (enkel alfanumerieke tekens, underscores zijn toegelaten maar spaties niet) en "label/score" vervang je door je eigen labels/scores (enkel alfanumerieke tekens). In het geval van een multilabel-dataset scheid je de verschillende labels door een puntkomma (**zonder** spatie erna). Gevallen waarvoor geen annotatie beschikbaar is, duid je aan met "nvt" (zonder de aanhalingstekens). Ga tot slot naar onderstaande codecel en vervang 1) "[naam_dataset]" (in variabele `bestandsnaam`) door de naam van je xlsx-bestand en 2) "[naam_tabblad]" (in variabele `tabblad`) door de naam van het tabblad van het xlsx-bestand waarin de annotaties zich bevinden (te vinden onderaan in het bestand). Druk daarna op de "play"-knop rechtsboven in de cel.

In [ ]:
# WIJZIG DE WAARDES VAN ONDERSTAANDE VARIABELEN OP BASIS VAN JE DATASET
bestandsnaam = "[naam_dataset].xlsx"
tabblad = "[naam_tabblad]"

Klik vervolgens ook op het "play"-icoontje van de volgende codecel. Dit stukje code leest het bestand dat je zonet hebt opgeladen in, waarna het de annotaties op een gestructureerde manier opslaat in de variabele `d_dataset`. Ter illustratie worden de eerste tien items uit de dataset geprint op je scherm.

In [ ]:
from openpyxl import load_workbook


# xlsx-bestand en tabblad inlezen
wb = load_workbook(bestandsnaam)
ws = wb[tabblad]

# berekenen hoeveel annotatoren er zijn
rij_1 = ws[1]
aantal_annotatoren = len([cel.value for cel in rij_1 if "ANNOTATOR" in cel.value])

# verifiëren dat er effectief slechts twee annotatoren zijn als Cohen's kappa tussen de IAA-metrics zit
if "cohens_kappa" in l_metrics:
  assert aantal_annotatoren == 2, f"Je hebt aangeduid dat er slechts twee annotatoren zijn, maar de dataset bevat er meer."

# rij per rij door dataset gaan en gegevens op een gestructureerde manier opslaan in `d_dataset`
d_dataset = {}

for row in ws.iter_rows(min_row=2, max_col=(aantal_annotatoren + 1)):
  id_item = row[0].value
  assert id_item not in d_dataset, f"De dataset bevat twee items met volgende ID: {id_item}."
  d_dataset[id_item] = {}

  for loop in range(aantal_annotatoren):
    id_annotator = loop + 1
    annotatie = row[id_annotator].value
    d_dataset[id_item][id_annotator] = str(annotatie)

# eerste tien items uit de dataset printen op het scherm
print(f"Eerste tien items uit de dataset (in gestructureerd formaat): {list(d_dataset.items())[:10]}")

## **Berekenen van IAA**
Nu ook onze geannoteerde data is ingelezen, kunnen we overgaan tot de laatste stap: het berekenen van de IAA-scores. Hiervoor moeten we eerst onderstaande **twee** cellen activeren door opnieuw op de "play"-knop ervan te klikken. Deze code zullen we zo meteen gebruiken om de uiteindelijke IAA-waardes te berekenen voor elke *metric* in de lijst met geschikte *metrics* die we eerder in deze notebook gedefinieerd hebben.

In [ ]:
!pip install krippendorff

In [ ]:
import krippendorff
from nltk.metrics import agreement, jaccard_distance
import numpy as np
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import aggregate_raters, fleiss_kappa
from typing import Optional


def iaa_berekenen(
    l_metrics: list, d_dataset: dict, multilabel: bool, per_label: bool, soort_ordinaal: Optional[str], aantal_annotatoren: int
  ) -> dict:
  soort_ordinaal = "nominal" if soort_ordinaal is None else soort_ordinaal
  d_iaa = {}

  if not per_label:

    for metric in l_metrics:

      if metric == "cohens_kappa":
        iaa_score = cohen_kappa_score([d_dataset[item][1] for item in d_dataset], [d_dataset[item][2] for item in d_dataset])

      if metric == "fleiss_kappa":
        dataset_converted = aggregate_raters([[d_dataset[item][id_annotator] for id_annotator in d_dataset[item]] for item in d_dataset])[0]
        iaa_score = fleiss_kappa(dataset_converted)

      if metric == "krippendorffs_alfa":

        if not multilabel:
          dataset_converted = []

          for loop in range(aantal_annotatoren):
            id_annotator = loop + 1
            dataset_converted.append(" ".join([str(d_dataset[item][id_annotator]) for item in d_dataset]))

          dataset_converted = [[np.nan if v == "nvt" else float(v) for v in id_annotator.split()] for id_annotator in dataset_converted]
          iaa_score = krippendorff.alpha(reliability_data=dataset_converted, level_of_measurement=soort_ordinaal)

        else:
          dataset_converted = [(id_annotator, item, frozenset(d_dataset[item][id_annotator].split(";"))) for item in d_dataset for id_annotator in d_dataset[item]]
          task = agreement.AnnotationTask(dataset_converted, distance=jaccard_distance)
          iaa_score = task.alpha()

      d_iaa[metric] = round(iaa_score, 4) if iaa_score is not None else None

  else:
    l_labels = []

    for item in d_dataset:

      for id_annotator in d_dataset[item]:

        for label in d_dataset[item][id_annotator].split(";"):

          if label not in l_labels:
            l_labels.append(label)

    for label in sorted(l_labels):
      d_iaa[label] = {}
      dataset_binary = {item: {id_annotator: 1 if str(label) in d_dataset[item][id_annotator].split(";") else 0 for id_annotator in d_dataset[item]} for item in d_dataset}

      for metric in l_metrics:

        if metric == "cohens_kappa":
          iaa_score = cohen_kappa_score([dataset_binary[item][1] for item in dataset_binary], [dataset_binary[item][2] for item in dataset_binary])

        if metric == "fleiss_kappa":
          dataset_binary_converted = aggregate_raters([[dataset_binary[item][id_annotator] for id_annotator in dataset_binary[item]] for item in dataset_binary])[0]
          iaa_score = fleiss_kappa(dataset_binary_converted)

        if metric == "krippendorffs_alfa":
          dataset_binary_converted = []

          for loop in range(aantal_annotatoren):
            id_annotator = loop + 1
            dataset_binary_converted.append(" ".join([str(dataset_binary[item][id_annotator]) for item in dataset_binary]))

          dataset_binary_converted = [[np.nan if v == "nvt" else float(v) for v in id_annotator.split()] for id_annotator in dataset_binary_converted]
          iaa_score = krippendorff.alpha(reliability_data=dataset_binary_converted, level_of_measurement=soort_ordinaal)

        d_iaa[label][metric] = round(iaa_score, 4) if iaa_score is not None else None

  return d_iaa

Tot slot gebruiken we de code uit bovenstaande cel om de IAA-waardes te berekenen en printen we de resultaten op het scherm. Hiervoor dien je op de "play"-knop te klikken van de cel helemaal onderaan in de notebook. Scores voor Cohen's κ en Fleiss' κ liggen tussen -1 en 1: **scores lager dan nul** geven aan dat er **minder overeenkomst** is tussen de annotatoren dan te verwachten valt dan bij toeval, een **score van nul** geeft aan dat de **overeenkomst even groot is als bij toeval**, en **scores hoger dan nul** geven aan dat er **meer overeenkomst** is tussen de annotatoren dan te verwachten valt bij toeval (met een score van één die betekent dat de overeenkomst unaniem is). Om κ-waarden te interpreteren wordt vaak ook onderstaande schaal van [Landis en Koch (1977)](https://pubmed.ncbi.nlm.nih.gov/843571/) gebruikt (Tabel 2), al moeten we hierbij wel benadrukken dat deze schaal eerder een intuïtieve interpretatie voorstelt dan een volwaardige, algemeen aanvaarde wetenschappelijke analyse.

|Bereik      |Interpretatie             |
|:----------:|:------------------------:|
|< 0         |Poor agreement            |
|0.01 - 0.2  |Slight agreement          |
|0.21 - 0.4  |Fair agreement            |
|0.41 - 0.6  |Moderate agreement        |
|0.61 - 0.8  |Substantial agreement     |
|0.81 - 1    |(Almost) perfect agreement|

*Tabel 2: Interpretatie κ-metrics volgens Landis en Koch (1977), originele bewoordingen.*

Voor Krippendorff's α - die ook een theoretisch bereik heeft van -1 tot 1 - wordt gewoonlijk een andere interpretatie met striktere drempelwaarden gehanteerd, voorgesteld door Klaus Krippendorff zelf in diens boek "[Content Analysis: An Introduction to Its Methodology](https://doi.org/10.4135/9781071878781)". Het overzicht van deze interpretatie wordt weergegeven in Tabel 3.

|Bereik    |Interpretatie                       |
|:--------:|:----------------------------------:|
|< 0.67    |Conclusions should be discounted    |
|0.67 - 0.8|Acceptable for tentative conclusions|
|0.81 - 1  |Acceptable for definite conclusions |

*Tabel 3: Interpretatie α volgens Krippendorff, originele bewoordingen.*

In [ ]:
# IAA berekenen door middel van code uit vorige codecel
d_iaa = iaa_berekenen(l_metrics, d_dataset, multilabel, per_label, soort_ordinaal, aantal_annotatoren)


# IAA printen
def iaa_printen_op_scherm(d_iaa: dict, per_label: bool) -> None:
  d_mapping = {"cohens_kappa": "Cohen's kappa", "fleiss_kappa": "Fleiss' kappa", "krippendorffs_alfa": "Krippendorff's alfa"}

  if not per_label:

    for metric in d_iaa:
      print(f"De inter-annotator agreement voor {d_mapping[metric]} bedraagt {d_iaa[metric]}.\n")

  else:

    for label in d_iaa:
      print(f"Inter-annotator agreement voor label '{label}':")

      for metric in d_iaa[label]:
        print(f"\t- {d_mapping[metric]} = {d_iaa[label][metric]}")

      print("\n")


iaa_printen_op_scherm(d_iaa, per_label)

## **Credits**
- **Auteur**: Jasper Degraeuwe
- **Licentie**: [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/)

<figure>
<img src="https://drive.google.com/uc?export=download&id=1g1vHFB7UJhnVOPg0EVirQcxAKndVnprP" width="200" height="160">
</figure>